In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

DIR_PATH_TOM = "C:\\Users\\ttomn\\OneDrive\\Desktop\\data-science-transportation-research"

In [2]:
file_name = "bin_removed_and_streets_data_and_speed_limits_and_humps_and_turn_traffic_calming_and_intersections_improvements_and_leading_pedestrian_and_summons_and_poi_and_collisions_and_streets.csv"

In [3]:
df = pd.read_csv(f"{DIR_PATH_TOM}\\{file_name}")

C:\Users\ttomn\AppData\Local\Temp\ipykernel_13896\795797161.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{DIR_PATH_TOM}\\{file_name}")


In [4]:
# df = df.drop(columns=["Unnamed: 0"])
some_columns = df[['ST_INDEX', 'the_geom', 'YEAR', 'SUMMONS', 'ST_WIDTH', 'ST_LENGTH', "IS_SIGNED"]]
df = df.drop(columns=["Unnamed: 0", 'ST_INDEX', 'the_geom', 'YEAR', 'SUMMONS', 'ST_WIDTH', 'ST_LENGTH', "IS_SIGNED"])

In [6]:
df = df.astype(np.uint8)

In [9]:
df = pd.concat([df, some_columns], axis=1)

In [10]:
df.to_csv("final_df_with_unit8.csv")

In [ ]:
y = df['COLLISIONS']
X = df.drop('COLLISIONS', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)

MemoryError: Unable to allocate 2.06 GiB for an array with shape (17, 16229410) and data type float64

In [ ]:
model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
result = model.fit()

# Print the model summary
print(result.summary())




In [13]:
df.head(10)

,MONTH,YEAR,COLLISIONS,Residential,Education,Cultural,Recreational,Social Services,Transportation,Commercial,...,Bridge,Tunnel,Boardwalk,PathTrail,StepStreet,Driveway,Ramp,Alley,U-Turn,FerryRoute
0,1,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,6,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
6,7,2012,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
7,8,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
8,9,2012,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
9,10,2012,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df.dtypes

MONTH                    uint8
COLLISIONS               uint8
Residential              uint8
Education                uint8
Cultural                 uint8
Recreational             uint8
Social Services          uint8
Transportation           uint8
Commercial               uint8
Government               uint8
Religious Institution    uint8
Health Services          uint8
Public Safety            uint8
Water                    uint8
Miscellaneous            uint8
PEDESTRIAN_BINARY        uint8
INTERSECTIONS_BINARY     uint8
TURN_TRAFFIC_BINARY      uint8
HUMPS                    uint8
SPEED_LIMIT              uint8
With                     uint8
Against                  uint8
Two-Way                  uint8
Street                   uint8
Highway                  uint8
Bridge                   uint8
Tunnel                   uint8
Boardwalk                uint8
PathTrail                uint8
StepStreet               uint8
Driveway                 uint8
Ramp                     uint8
Alley   